## Video Load

In [3]:
# !pip install opencv-python
# !pip install matplotlib==3.3.0

In [4]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [5]:
def load_video(path, ms=100):
    
    # Video Capture 객체 생성
    capture = cv2.VideoCapture(path)
    
    # Frame을 저장할 List 선언
    frames = []
    
    while capture.isOpened(): # Video Capture가 준비되었는지 확인
        
        run, frame = capture.read() # 다음 Frame 읽기
        
        if run: # Frame을 읽은 경우
            cv2.imshow("video", frame)
            if cv2.waitKey(ms) & 0xFF == ord('q'):
                break
        else: # 재생이 완료되어 더 이상 Frame을 읽을 수 없는 경우
            break
        
        # Frame List에 추가
        frames.append(frame)

    capture.release() # Capture 자원 반납
    cv2.destroyAllWindows() # 창 제거
    
    return np.array(frames, dtype='uint8')

In [6]:
# Video가 저장된 경로 입력
PATH = r".\pushup_sample.mp4"
# PATH = r"C:\Users\admin\Desktop\KUDIP\Video Samples\Hand Video2.mov"
# PATH = r"C:\Users\hj\AICV\수업\디지털영상처리\Video Samples\highway.mov"
# PATH = r"C:\Users\hj\AICV\수업\디지털영상처리\Video Samples\earth.avi"

# Video 재생 및 반환 (Numpy Array)
video = load_video(PATH)

In [7]:
print(f"- Data Type: {type(video)}")
print(f"- Data Shape: {video.shape} *Frames x Height x Width x Channel")
if video.size > 0:
    print(f"- Maximum Intensity: {video.max()}")
    print(f"- Minimum Intensity: {video.min()}")
else:
    print("The array is empty, so maximum and minimum intensities cannot be determined.")

- Data Type: <class 'numpy.ndarray'>
- Data Shape: (12, 360, 640, 3) *Frames x Height x Width x Channel
- Maximum Intensity: 97
- Minimum Intensity: 0


In [153]:
import cv2
import numpy as np

def calculate_circularity(contour):
    area = cv2.contourArea(contour)
    perimeter = cv2.arcLength(contour, True)
    circularity = (4 * np.pi * area) / (perimeter ** 2)
    return circularity

def find_largest_circularity_contour(contours):
    max_circularity = -1
    largest_contour = None
    
    for contour in contours:
        circularity = calculate_circularity(contour)
        if circularity > max_circularity:
            max_circularity = circularity
            largest_contour = contour
    
    return largest_contour

def find_center_of_contour(contour):
    ((x, y), _radius) = cv2.minEnclosingCircle(contour)
    center = (int(x), int(y))
    return center

def high_boost_filter(image, kernel_size=3, k=1.2):
    """
    이미지에 High Boost Filtering을 적용합니다.

    Args:
        image (numpy.ndarray): 입력 이미지
        kernel_size (int): 커널 크기 (기본값은 3)
        k (float): High Boost 계수 (기본값은 1.2)

    Returns:
        numpy.ndarray: High Boost Filtering된 이미지
    """
    # 평균 필터링을 통해 저주파 성분 추출
    blur = cv2.blur(image, (kernel_size, kernel_size))

    # High Boost Filtering 적용
    high_boost_image = image * k - blur

    # 결과 이미지 반환
    return high_boost_image

def preprocess_frame(frame):
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    intensity = hsv[:, :, 2]

    # boosted = high_boost_filter(intensity)

    # 원형 커널 정의 (반지름 5)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    opening = cv2.morphologyEx(intensity, cv2.MORPH_DILATE, kernel)

    hsv[:, :, 2] = opening
    result = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

    return result


def count_pushups(head_positions):
    count = 0
    state = 0  # 0: 시작, 1: 내려감, 2: 올라옴
    for i in range(1, len(head_positions)):
        if head_positions[i-1] is None or head_positions[i] is None:
            continue
        if state == 0 and head_positions[i][1] > head_positions[i-1][1]:
            state = 1
        elif state == 1 and head_positions[i][1] < head_positions[i-1][1]:
            count += 1
            state = 2
        elif state == 2 and head_positions[i][1] > head_positions[i-1][1]:
            state = 1
    return count

def detect_skin(img):

    #converting from gbr to hsv color space
    img_HSV = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    #skin color range for hsv color space 
    HSV_mask = cv2.inRange(img_HSV, (0, 15, 0), (17,170,255)) 

    HSV_mask = cv2.morphologyEx(HSV_mask, cv2.MORPH_OPEN, np.ones((3,3), np.uint8))

    #converting from gbr to YCbCr color space
    img_YCrCb = cv2.cvtColor(img, cv2.COLOR_BGR2YCrCb)

    #skin color range for hsv color space 
    YCrCb_mask = cv2.inRange(img_YCrCb, (0, 135, 85), (255,180,135)) 
    YCrCb_mask = cv2.morphologyEx(YCrCb_mask, cv2.MORPH_OPEN, np.ones((3,3), np.uint8))

    #merge skin detection (YCbCr and hsv)
    global_mask=cv2.bitwise_and(YCrCb_mask,HSV_mask)
    global_mask=cv2.medianBlur(global_mask, 5)
    global_mask = cv2.morphologyEx(global_mask, cv2.MORPH_OPEN, np.ones((7,7), np.uint8))


    HSV_result = cv2.bitwise_not(HSV_mask)
    YCrCb_result = cv2.bitwise_not(YCrCb_mask)
    global_result=cv2.bitwise_not(global_mask)

    return global_mask

def get_ground(image):
    height, width = image.shape

    for y in range(height-1, -1, -1):
        count = 0
        for x in range(width):
            if image[y, x] == 255:
                count += 1
                if count >= 10:
                    return y  # Return the position of the first continuous 1s
            else:
                count = 0
    
    return None



In [154]:
import cv2
import numpy as np

def three_step_search(prev_frame, curr_frame, block_size=16, search_range=7):
    """
    3 단계 탐색 알고리즘을 사용하여 모션 벡터를 계산합니다.

    Args:
        prev_frame (numpy.ndarray): 이전 프레임
        curr_frame (numpy.ndarray): 현재 프레임
        block_size (int): 블록 크기 (기본값은 16)
        search_range (int): 탐색 범위 (기본값은 7)

    Returns:
        numpy.ndarray: 모션 벡터 배열
    """
    height, width = prev_frame.shape[:2]
    motion_vectors = np.zeros((height // block_size, width // block_size, 2), dtype=np.int32)

    for y in range(0, height, block_size):
        for x in range(0, width, block_size):
            # 이전 프레임의 블록 추출
            prev_block = prev_frame[y:y+block_size, x:x+block_size]

            # 현재 프레임에서의 최적의 블록 위치 찾기
            best_match = None
            best_sad = float('inf')

            # 1단계: 중심에서 ±search_range 범위 내에서 탐색
            for dy in range(-search_range, search_range+1, 2*search_range):
                for dx in range(-search_range, search_range+1, 2*search_range):
                    curr_y, curr_x = y + dy, x + dx
                    if 0 <= curr_y < height-block_size and 0 <= curr_x < width-block_size:
                        curr_block = curr_frame[curr_y:curr_y+block_size, curr_x:curr_x+block_size]
                        sad = np.sum(np.abs(prev_block - curr_block))
                        if sad < best_sad:
                            best_sad = sad
                            best_match = (dy, dx)

            if best_match is None:
                return None
            # 2단계: 최적 위치에서 ±search_range//2 범위 내에서 탐색
            for dy in range(best_match[0]-search_range//2, best_match[0]+search_range//2+1, search_range//2):
                for dx in range(best_match[1]-search_range//2, best_match[1]+search_range//2+1, search_range//2):
                    curr_y, curr_x = y + dy, x + dx
                    if 0 <= curr_y < height-block_size and 0 <= curr_x < width-block_size:
                        curr_block = curr_frame[curr_y:curr_y+block_size, curr_x:curr_x+block_size]
                        sad = np.sum(np.abs(prev_block - curr_block))
                        if sad < best_sad:
                            best_sad = sad
                            best_match = (dy, dx)

            # 3단계: 최적 위치에서 ±1 범위 내에서 탐색
            for dy in range(best_match[0]-1, best_match[0]+2, 1):
                for dx in range(best_match[1]-1, best_match[1]+2, 1):
                    curr_y, curr_x = y + dy, x + dx
                    if 0 <= curr_y < height-block_size and 0 <= curr_x < width-block_size:
                        curr_block = curr_frame[curr_y:curr_y+block_size, curr_x:curr_x+block_size]
                        sad = np.sum(np.abs(prev_block - curr_block))
                        if sad < best_sad:
                            best_sad = sad
                            best_match = (dy, dx)

            motion_vectors[y//block_size, x//block_size] = best_match

    return motion_vectors

def calculate_angle(points):
    if len(points) < 2:
        return None
    # Fit a line to the points
    [vx, vy, x, y] = cv2.fitLine(np.array(points), cv2.DIST_L2, 0, 0.01, 0.01)
    # Calculate the angle with the horizontal axis
    angle = np.arctan2(vy[0], vx[0]) * 180 / np.pi
    return angle

def draw_flow(img, flow, step=16):
    h, w = img.shape[:2]
    y, x = np.mgrid[step//2:h:step, step//2:w:step].reshape(2, -1)
    fx, fy = flow[y, x].T

    lines = np.vstack([x, y, x+fx, y+fy]).T.reshape(-1, 2, 2)
    lines = np.int32(lines + 0.5)
    vis = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    vis = cv2.cvtColor(vis, cv2.COLOR_GRAY2BGR)
    cv2.polylines(vis, lines, 0, (0, 255, 0))

    for (x1, y1), (x2, y2) in lines:
        cv2.circle(vis, (x1, y1), 1, (0, 255, 0), -1)
    return vis

def get_vertical_motion_coords(flow, threshold=1.0, v_speed=1.0):
    h, w = flow.shape[:2]
    coords = []
    for y in range(h):
        for x in range(w):
            if abs(flow[y, x, 1] - v_speed) < threshold:
                coords.append((x, y))
    return coords



In [410]:
import math

def main(video_path):
    cap = cv2.VideoCapture(video_path)

    ret, prev_frame = cap.read()
    h, w, _ = prev_frame.shape
    # prev_frame = prev_frame[h//3:, w//4-20:w*3//4+20]
    head_positions = []
    frames = []

    prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
    
    bg_subtractor = cv2.createBackgroundSubtractorMOG2()


    while cap.isOpened():
        ret, frame = cap.read()
        h, w, _ = frame.shape
        # frame = frame[h//3:, w//4-20:w*3//4+20]

        if not ret:
            break

        cur_frame = preprocess_frame(frame)
        gray = cv2.cvtColor(cur_frame, cv2.COLOR_BGR2GRAY)
        _, otsu_threshold = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
        cv2.imshow('otsu', otsu_threshold)

        
        gray = cv2.cvtColor(cur_frame, cv2.COLOR_BGR2GRAY)
        

        # 배경 차분 수행
        fg_mask = bg_subtractor.apply(cur_frame)
        
        # 객체 추출
        contours, _ = cv2.findContours(fg_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        # 움직이는 객체가 사람인지 판단하고 선 그리기
        for contour in contours:
            area = cv2.contourArea(contour)
            if area > 1000:  # 임계값 설정
                x, y, w, h = cv2.boundingRect(contour)
                cv2.rectangle(cur_frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        
        edges = cv2.Canny(otsu_threshold, 100, 200)
        

        # Calculate optical flow
        flow = cv2.calcOpticalFlowFarneback(prev_gray, gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)

        # 수직 방향 모션 벡터 추출
        flow_y = flow[..., 1]

        # 수직 방향 모션 벡터의 크기 계산
        magnitude = np.abs(flow_y)

        # 임계값 이상의 크기를 가진 구역 식별
        threshold = 12  # 임의의 임계값
        motion_region = np.where(magnitude > threshold)

        # 각 구역의 중심점 계산
        if motion_region[1].size > 0:
            x_coords = np.mean(motion_region[1])
            cv2.line(cur_frame, (int(x_coords), 0), (int(x_coords), h), (255, 0, 0), thickness=3)

        cv2.imshow('edges', edges)
        cv2.imshow('Result', cur_frame)


        

        # 이전 프레임 업데이트
        prev_gray = gray.copy()

        # frames.append(cur_frame)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()
    
    pushup_count = count_pushups(head_positions)
    print(f"Total push-ups: {pushup_count}")

In [411]:
if __name__ == "__main__":
    video_path = "pushup_sample.mp4"
    main(video_path)

Total push-ups: 0


In [395]:

# MOG2 배경 제거 모델 생성
fgbg = cv2.createBackgroundSubtractorMOG2()

# 영상 읽기
cap = cv2.VideoCapture('./pushup_sample.mp4')

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    # 배경 제거 수행
    fgmask = fgbg.apply(frame)
    
    # 객체 추출
    contours, _ = cv2.findContours(fgmask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # 움직이는 객체가 사람인지 판단하고 선 그리기
    for contour in contours:
        area = cv2.contourArea(contour)
        if area > 800:  # 임계값 설정
            x, y, w, h = cv2.boundingRect(contour)
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
    
    # 결과 출력
    cv2.imshow('Object Detection', frame)
    
    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [399]:
import cv2

# 배경 제거 모델 생성
fgbg = cv2.createBackgroundSubtractorMOG2()

# 영상 읽기
cap = cv2.VideoCapture('./pushup_sample.mp4')

# 푸쉬업을 하는 사람 수 초기화
pushup_count = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    # 배경 제거 수행
    fgmask = fgbg.apply(frame)
    
    # 객체 추출
    contours, _ = cv2.findContours(fgmask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # 푸쉬업을 하는 사람 감지 및 카운트
    for contour in contours:
        area = cv2.contourArea(contour)
        if area > 500:  # 임계값 설정
            x, y, w, h = cv2.boundingRect(contour)
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            
            # 푸쉬업을 하는 사람으로 판단하는 추가적인 조건을 여기에 추가할 수 있습니다.
            # 예를 들어, 객체의 높이와 가로 세로 비율 등을 고려하여 판단할 수 있습니다.
            # 여기서는 간단히 객체의 개수를 세어서 푸쉬업을 하는 사람의 수를 계산합니다.
            pushup_count += 1
    
    # 결과 출력
    cv2.putText(frame, f'Pushup Count: {pushup_count}', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    cv2.imshow('Pushup Detection', frame)
    
    if cv2.waitKey(20) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
